# DC_TTS - a test on the pre-trained model which is based on the LJ speech dataset

**!!** **Remember to activate Hardware acceleration** **!!**
Edit → Notebook Settings → select GPU from the hardware accelerator menu

This notebook allows you to synthesize English sentences with a pre-trained model based on the LJ speech dataset.
After doing some preliminary steps, you will be able to choose your own sentences that should be synthesized or stick with the sentences pre-defined in the [harvard_sentences](http://www.cs.columbia.edu/~hgs/audio/harvard.html).txt. At the end you will be able to use a simple audio widget to play the synthesized files.

In [ ]:
# clone the repository
!git clone https://github.com/jkuhlemann/dc_tts.git

Cloning into 'dc_tts'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 238 (delta 42), reused 15 (delta 6), pack-reused 168
Receiving objects: 100% (238/238), 3.15 MiB | 14.10 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [ ]:
# create folders necessary for everything to work
!mkdir /content/dc_tts/samples
!mkdir /content/dc_tts/logdir
!mkdir /data/
!mkdir /data/private/
!mkdir /data/private/voice/

In [ ]:
# before going further, we have to change some lines in
# hyperparams.py, so that it loads the right directories
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/dc_tts/hyperparams.py"

orig_1 = "test_data = 'harvard_sentences.txt'"
repl_1 = "test_data = '/content/dc_tts/harvard_sentences.txt'"

orig_2 = 'logdir = "logdir/LJ01"'
repl_2 = 'logdir = "/content/dc_tts/logdir/LJ01"'

orig_3 = "sampledir = 'samples'"
repl_3 = "sampledir = '/content/dc_tts/samples'"

replacement(file, orig_1, repl_1)
replacement(file, orig_2, repl_2)
replacement(file, orig_3, repl_3)

In [ ]:
# create a txt in case user wants to choose own sentences
f = open("/content/dc_tts/own_sentences.txt", 'w')

In [ ]:
# download pre-trained model
import urllib.request

url = "https://www.dropbox.com/s/1oyipstjxh2n5wo/LJ_logdir.tar?dl=1"  
u = urllib.request.urlopen(url)
data = u.read()
u.close()
with open('/content/dc_tts/LJ_logdir.tar', "wb") as f :
    f.write(data)

In [ ]:
# extract the pre-trained model
import zipfile
with zipfile.ZipFile('/content/dc_tts/LJ_logdir.tar', 'r') as zip_ref:
    zip_ref.extractall('/content/dc_tts/')

In [ ]:
# move the pre-trained model data to logdir folder
!mv /content/dc_tts/LJ01-1 /content/dc_tts/logdir
!mv /content/dc_tts/LJ01-2 /content/dc_tts/logdir

In [ ]:
# function to use own sentences for TTS
# with this, you can add as many sentences as you wish for the synthesis
def sentences_to_synthesize():
  x = input("Do you want to use the example sentences or synthesize your own sentences? Please choose(example/own): ")
  if x == "example":
    orig_1 = "test_data = 'harvard_sentences.txt'"
    repl_1 = "test_data = '/content/dc_tts/harvard_sentences.txt'"
    replacement(file, orig_1, repl_1)
    print("Good. Just go to the next cell then.")  
    pass
  elif x == "own":
    orig_1 = "test_data = '/content/dc_tts/harvard_sentences.txt'"
    repl_1 = "test_data = '/content/dc_tts/own_sentences.txt'"
    replacement(file, orig_1, repl_1)
    n = 1
    with open('/content/dc_tts/own_sentences.txt', 'w') as f:
      f.write('The code ignores the first line in the .txt. This is just a line so that everything works \n')

    own_sentence = input("Alright. Please write your sentence here: ")
    own_sentence = f"{n}. {own_sentence} \n"
    
    with open('/content/dc_tts/own_sentences.txt', 'a') as f:
      f.write(own_sentence)
  while True:
    more = input("Would you like to add another sentence? (y/n): ")
    if more == 'y':
      n = n + 1 
      own_sentence = input("Write your sentence here: ")
      own_sentence = f"{n}. {own_sentence} \n"
      with open('/content/dc_tts/own_sentences.txt', 'a') as f:
        f.write(own_sentence)
    elif more == 'n':
      return

  return
  
sentences_to_synthesize()

In [ ]:
# we have to manually switch to Tensorflow version 1.x since this is what the repo uses
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# synthesize the sentences specified in 'harvard_sentences.txt.' (Your own or pre-defined)
# you will find the resulting .wav files in /content/dc_tts/samples 
# or use the widget implementation below to play the audio
!python /content/dc_tts/synthesize.py

In [ ]:
from IPython.display import Audio, display
# function to play desired audiofile
# choose the desired file for playback (ascending numbers from 1)
def play_audio():
  numb = input("Which audiofile would you like to listen to? (Choose number): ")
  filepath = f"/content/dc_tts/samples/{numb}.wav"
  return Audio(filepath)

play_audio()

That's it! We hope you enjoyed testing the TTS model!